In [1]:
!pip install langchain-community langchain langchain-core langchain-openai

In [3]:
!pip install -qU transformers huggingface_hub

In [2]:
!pip install -qU weaviate-client faiss-cpu chromadb pinecone

In [4]:
!pip install pypdf

In [5]:
pip install -U weaviate-client

# ***Data Loading***

In [6]:
from langchain_community.document_loaders import PyPDFLoader

In [7]:
from langchain.document_loaders import PyPDFLoader

In [8]:
loader=PyPDFLoader("/content/heart_disease.pdf",extract_images=True)
document=loader.load()

In [9]:
len(document)

8

In [43]:
document[2].page_content

'• The patient may experience difficulty in breathing or shortness of breath. • Nausea . • Light-headedness or fainting. • Sweating. • Women are more likely to experience shortness of breath, nausea, vomiting, and back or jaw pain.  Symptoms of a Stroke Include: • Sudden weakness of the face, arm, or leg, most often on one side of the body. • Numbness of the face, arm, or leg, especially on one side of the body. • Difficulty in speaking or understanding speech. • Difficulty in seeing with one or both eyes. • Difficulty in walking, dizziness, loss of balance, or coordination. • Severe headache with no known cause. • Fainting or loss of consciousness.  Symptoms of Rheumatic Heart Disease: • Symptoms of rheumatic heart disease include: shortness of breath, fatigue, irregular heartbeats, chest pain, and fainting. • Symptoms of rheumatic fever include: fever, pain and swelling of the joints, nausea, stomach cramps, and vomiting.  Prevention: You can help prevent cardiovascular disease by ma

# ***Creating Text Chunks***

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

In [10]:
text_chunks=text_splitter.split_documents(document)

In [12]:
len(text_chunks)

23

# Embedding model

In [11]:
import os
from google.colab import userdata

In [12]:
os.environ["huggingface_api_token"]=userdata.get("HF_Token")

In [13]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [14]:
hf_embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/tmp/ipython-input-3841940515.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or

In [61]:
hf_embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False, 'architecture': 'MPNetModel'})
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-mpnet-base-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

# DataIngestion Into ***Weaviate DB***

In [15]:
from langchain_community.vectorstores import Weaviate
import weaviate

In [16]:
from google.colab import userdata
WEAVIATE_API_KEY=userdata.get("WEAVIATE_API_KEY")
WEAVIATE_CLUSTER=userdata.get("WEAVIATE_CLUSTER")

In [17]:
import os
from weaviate.classes.init import Auth

In [18]:
client=weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_CLUSTER,
    auth_credentials=Auth.api_key(WEAVIATE_API_KEY)
)

In [23]:
client

In [19]:
!pip install langchain-weaviate

In [20]:
from langchain_weaviate.vectorstores import WeaviateVectorStore

In [21]:
weaviate_vectorstore=WeaviateVectorStore.from_documents(text_chunks,embedding=hf_embedding,client=client)

ERROR:weaviate-client:{'message': 'Failed to send all objects in a batch of 10', 'error': 'WeaviateInsertManyAllFailedError("Every object failed during insertion. Here is the set of all errors: \'aapl:keywords\' is not a valid property name. Property names in Weaviate are restricted to valid GraphQL names, which must be “/[_A-Za-z][_0-9A-Za-z]{0,230}/”")'}
ERROR:weaviate-client:{'message': 'Failed to send 10 objects in a batch of 10. Please inspect client.batch.failed_objects or collection.batch.failed_objects for the failed objects.'}
ERROR:weaviate-client:{'message': 'Failed to send all objects in a batch of 10', 'error': 'WeaviateInsertManyAllFailedError("Every object failed during insertion. Here is the set of all errors: \'aapl:keywords\' is not a valid property name. Property names in Weaviate are restricted to valid GraphQL names, which must be “/[_A-Za-z][_0-9A-Za-z]{0,230}/”")'}
ERROR:weaviate-client:{'message': 'Failed to send 10 objects in a batch of 10. Please inspect clien

In [27]:
weaviate_vectorstore

In [22]:
weaviate_retriever=weaviate_vectorstore.as_retriever(search_kwargs={"k":4})

# Data Ingestion into ***Pinecone DB***

In [23]:
from langchain_community.embeddings import OpenAIEmbeddings

In [24]:
os.environ["OPENAI_API_KEY"]=userdata.get("OPENAI_API_KEY")

In [25]:
openai_embeddings=OpenAIEmbeddings()

/tmp/ipython-input-1904077128.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  openai_embeddings=OpenAIEmbeddings()


In [26]:
!pip install -qU pinecone

In [27]:
!pip install -qU langchain-pinecone

In [28]:
from pinecone import Pinecone, ServerlessSpec
import pinecone
from langchain_pinecone import PineconeVectorStore

In [29]:
os.environ["PINECONE_API_KEY"]=userdata.get("pinecone_api_key")

In [30]:
pc=Pinecone()

In [31]:
pc.list_indexes()

[
    {
        "name": "rag",
        "metric": "cosine",
        "host": "rag-yxyq80x.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "vector_type": "dense",
        "dimension": 1536,
        "deletion_protection": "disabled",
        "tags": {
            "embedding_model": "text-embedding-3-small"
        }
    }
]

In [32]:
if "rag" not in pc.list_indexes().names():
  pc.create_index(
      name="rag",
      dimension=1536,
      metric="cosine",
      spec=ServerlessSpec(cloud="aws",region="us-east-1")
  )

In [33]:
pinecone_vectorstore=PineconeVectorStore.from_existing_index("rag",embedding=openai_embeddings)

In [91]:
pinecone_vectorstore=PineconeVectorStore.from_documents(
    text_chunks,
    embedding=openai_embeddings,
    index_name="rag"
)

In [41]:
pinecone_vectorstore

In [34]:
pinecone_retriever=pinecone_vectorstore.as_retriever(search_kwargs={"k":4})

# Combining both Retrievers

In [35]:
from langchain.retrievers.merger_retriever import MergerRetriever

In [36]:
final_retriver=MergerRetriever(retrievers=[pinecone_retriever,weaviate_retriever])

In [45]:
result=final_retriver.get_relevant_documents("Heart Disease Caused by Heart Infections")

/tmp/ipython-input-3986214792.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result=final_retriver.get_relevant_documents("Heart Disease Caused by Heart Infections")


In [46]:
result[0].page_content

'and fainting.  Heart Disease Caused by Heart Infections: There are three types of heart infections: • Pericarditis (inflammation of the pericardium, which is the fibrous sac surrounding the heart). • Myocarditis (inflammation of the myocardium, which is the thick middle layer of the heart muscle). • Endocarditis (inflammation of the endocardium, which is the inner lining of your heart chambers and heart valves). Heart infection symptoms can include: • Fever. • Shortness of breath. • Fatigue. •'

In [47]:
len(result)

4

In [37]:
from langchain.document_transformers import EmbeddingsRedundantFilter, LongContextReorder
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain.retrievers import ContextualCompressionRetriever

In [38]:
filter=EmbeddingsRedundantFilter(embeddings=openai_embeddings)
reordering=LongContextReorder()
pipeline=DocumentCompressorPipeline(transformers=[filter,reordering])

final_retriver_reranked=ContextualCompressionRetriever(
    base_compressor=pipeline, base_retriever=final_retriver, search_kwargs={"k":2, "include_metadata":True}
)

In [39]:
relevant_result=final_retriver_reranked.get_relevant_documents("Heart Disease Caused by Heart Infections")

/tmp/ipython-input-1553671539.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  relevant_result=final_retriver_reranked.get_relevant_documents("Heart Disease Caused by Heart Infections")


In [40]:
relevant_result[0].page_content

'Heart\tDisease\tIntroduction: Heart disease describes a range of conditions that affect the heart. Diseases under the umbrella term heart disease include: • Cardiovascular disease. • Heart arrhythmia. • Congenital heart disease.  • Cardiomyopathy. • Heart disease caused by heart infections. • Heart valve disease.  Symptoms:  Heart disease symptoms vary depending on which type of heart diseases you have.  Cardiovascular disease: Cardiovascular disease can result in narrowed or blocked blood vessels'

# Loading ***HuggingFace*** Model

In [1]:
!pip install -qU transformers

In [ ]:
!pip install langchain

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM,pipeline
from langchain.llms import HuggingFacePipeline

In [3]:
!hf auth login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `hf auth whoami` to get more information or `hf auth logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
The tok

In [1]:
model_name="meta-llama/Llama-3.2-3B"
model=AutoModelForCausalLM.from_pretrained(model_name)

NameError: name 'AutoModelForCausalLM' is not defined

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
tokenizer=AutoTokenizer.from_trained(model_name)

In [ ]:
pipe=pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=180,
    temperature=.6
)

In [ ]:
llm=HuggingFacePipeline(pipeline=pipe)

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
hf_qa=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=final_retriver_reranked,
    return_source_documents=True
)

In [41]:
!pip install -qU llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 18.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.6 MB/s eta 0:00:00


In [42]:
model_path="/content/drive/MyDrive/llama_models/zephyr-7b-beta.Q4_0.gguf"

In [44]:
from langchain.llms import LlamaCpp

In [45]:
model=LlamaCpp(
    streaming=True,
    model_path=model_path,
    max_tokens=1500,
    temperature=.5,
    top_p=1,
    gpu_layers=0,
    stream=True,
    verbose=True,
    n_threads=int(os.cpu_count()/2),
    n_ctx=4096
)

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3473: UserWarning: WARNING! gpu_layers is not default parameter.
                gpu_layers was transferred to model_kwargs.
                Please confirm that gpu_layers is what you intended.
  if (await self.run_code(code, result,  async_=asy)):
/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3473: UserWarning: WARNING! stream is not default parameter.
                stream was transferred to model_kwargs.
                Please confirm that stream is what you intended.
  if (await self.run_code(code, result,  async_=asy)):
llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /content/drive/MyDrive/llama_models/zephyr-7b-beta.Q4_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
ll

In [46]:
model

LlamaCpp(client=<llama_cpp.llama.Llama object at 0x7d332492f530>, model_path='/content/drive/MyDrive/llama_models/zephyr-7b-beta.Q4_0.gguf', n_ctx=4096, n_threads=1, max_tokens=1500, temperature=0.5, top_p=1.0, model_kwargs={'gpu_layers': 0, 'stream': True})

In [47]:
from langchain.chains import RetrievalQA

In [48]:
llama_qa=RetrievalQA.from_chain_type(
    llm=model,
    chain_type="stuff",
    retriever=final_retriver_reranked,
    return_source_documents=True
)

In [49]:
response=llama_qa.invoke("Heart Disease Caused by Heart Infections")

llama_perf_context_print:        load time =  143236.24 ms
llama_perf_context_print: prompt eval time =  143233.94 ms /   518 tokens (  276.51 ms per token,     3.62 tokens per second)
llama_perf_context_print:        eval time =   17847.40 ms /    30 runs   (  594.91 ms per token,     1.68 tokens per second)
llama_perf_context_print:       total time =  161113.76 ms /   548 tokens
llama_perf_context_print:    graphs reused =         76


In [51]:
print(response['result'])

 Endocarditis is a type of heart infection that causes inflammation of the inner lining of your heart chambers and heart valves.


In [52]:
response['query']

'Heart Disease Caused by Heart Infections'

In [54]:
print(response['source_documents'])

[_DocumentWithState(metadata={'aapl:keywords': '[]', 'creationdate': "D:20190823182446Z00'00'", 'creator': 'Word', 'keywords': '', 'moddate': "D:20190823182446Z00'00'", 'page': 0.0, 'page_label': '1', 'producer': 'macOS Version 10.14.6 (Build 18G87) Quartz PDFContext', 'source': '/content/heart_disease.pdf', 'title': 'Microsoft Word - أمراض القلب-ar-en-T-C.docx', 'total_pages': 8.0}, page_content='Heart\tDisease\tIntroduction: Heart disease describes a range of conditions that affect the heart. Diseases under the umbrella term heart disease include: • Cardiovascular disease. • Heart arrhythmia. • Congenital heart disease.  • Cardiomyopathy. • Heart disease caused by heart infections. • Heart valve disease.  Symptoms:  Heart disease symptoms vary depending on which type of heart diseases you have.  Cardiovascular disease: Cardiovascular disease can result in narrowed or blocked blood vessels', state={'embedded_doc': [-0.023324203956250143, 0.003611770180242625, 0.017754244607914037, -0.

In [55]:
for source in response['source_documents']:
  print(source.metadata)

{'aapl:keywords': '[]', 'creationdate': "D:20190823182446Z00'00'", 'creator': 'Word', 'keywords': '', 'moddate': "D:20190823182446Z00'00'", 'page': 0.0, 'page_label': '1', 'producer': 'macOS Version 10.14.6 (Build 18G87) Quartz PDFContext', 'source': '/content/heart_disease.pdf', 'title': 'Microsoft Word - أمراض القلب-ar-en-T-C.docx', 'total_pages': 8.0}
{'aapl:keywords': '[]', 'creationdate': "D:20190823182446Z00'00'", 'creator': 'Word', 'keywords': '', 'moddate': "D:20190823182446Z00'00'", 'page': 6.0, 'page_label': '7', 'producer': 'macOS Version 10.14.6 (Build 18G87) Quartz PDFContext', 'source': '/content/heart_disease.pdf', 'title': 'Microsoft Word - أمراض القلب-ar-en-T-C.docx', 'total_pages': 8.0}
{'aapl:keywords': '[]', 'creationdate': "D:20190823182446Z00'00'", 'creator': 'Word', 'keywords': '', 'moddate': "D:20190823182446Z00'00'", 'page': 0.0, 'page_label': '1', 'producer': 'macOS Version 10.14.6 (Build 18G87) Quartz PDFContext', 'source': '/content/heart_disease.pdf', 'titl

In [56]:
response=llama_qa.invoke("What are Symptoms of Heart Valve Disease?")

Llama.generate: 45 prefix-match hit, remaining 515 prompt tokens to eval
llama_perf_context_print:        load time =  143236.24 ms
llama_perf_context_print: prompt eval time =  130781.89 ms /   515 tokens (  253.95 ms per token,     3.94 tokens per second)
llama_perf_context_print:        eval time =   42528.98 ms /    71 runs   (  599.00 ms per token,     1.67 tokens per second)
llama_perf_context_print:       total time =  173386.95 ms /   586 tokens
llama_perf_context_print:    graphs reused =        115


In [57]:
response['query']

'What are Symptoms of Heart Valve Disease?'

In [61]:
response['result']

' Shortness of breath, particularly when the patient has been very active, chest pain after physical exertion, dizziness and fainting in advanced cases, fatigue and lethargy, heart palpitations, coughing up of blood, particularly in the case of mitral valve disease, swollen feet or ankles.'

In [63]:
print(response['source_documents'])

[_DocumentWithState(metadata={'aapl:keywords': '[]', 'creationdate': "D:20190823182446Z00'00'", 'creator': 'Word', 'keywords': '', 'moddate': "D:20190823182446Z00'00'", 'page': 2.0, 'page_label': '3', 'producer': 'macOS Version 10.14.6 (Build 18G87) Quartz PDFContext', 'source': '/content/heart_disease.pdf', 'title': 'Microsoft Word - أمراض القلب-ar-en-T-C.docx', 'total_pages': 8.0}, page_content='speech. • Difficulty in seeing with one or both eyes. • Difficulty in walking, dizziness, loss of balance, or coordination. • Severe headache with no known cause. • Fainting or loss of consciousness.  Symptoms of Rheumatic Heart Disease: • Symptoms of rheumatic heart disease include: shortness of breath, fatigue, irregular heartbeats, chest pain, and fainting. • Symptoms of rheumatic fever include: fever, pain and swelling of the joints, nausea, stomach cramps, and vomiting.  Prevention: You', state={'embedded_doc': [-0.019775957031061206, -0.005691792396869765, 0.010083909229060014, -0.02008

In [60]:
for source in response['source_documents']:
  print(source.metadata)

{'aapl:keywords': '[]', 'creationdate': "D:20190823182446Z00'00'", 'creator': 'Word', 'keywords': '', 'moddate': "D:20190823182446Z00'00'", 'page': 2.0, 'page_label': '3', 'producer': 'macOS Version 10.14.6 (Build 18G87) Quartz PDFContext', 'source': '/content/heart_disease.pdf', 'title': 'Microsoft Word - أمراض القلب-ar-en-T-C.docx', 'total_pages': 8.0}
{'aapl:keywords': '[]', 'creationdate': "D:20190823182446Z00'00'", 'creator': 'Word', 'keywords': '', 'moddate': "D:20190823182446Z00'00'", 'page': 0.0, 'page_label': '1', 'producer': 'macOS Version 10.14.6 (Build 18G87) Quartz PDFContext', 'source': '/content/heart_disease.pdf', 'title': 'Microsoft Word - أمراض القلب-ar-en-T-C.docx', 'total_pages': 8.0}
{'aapl:keywords': '[]', 'creationdate': "D:20190823182446Z00'00'", 'creator': 'Word', 'keywords': '', 'moddate': "D:20190823182446Z00'00'", 'page': 5.0, 'page_label': '6', 'producer': 'macOS Version 10.14.6 (Build 18G87) Quartz PDFContext', 'source': '/content/heart_disease.pdf', 'titl